In [ ]:
import json
import os

import tqdm

import src.visualization.fit_images as fi

In [ ]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

In [ ]:
DATASET_PATH = os.path.abspath("./data/raw/emptyNonEmptyDataset_ETL")
IMAGE_PATH = os.path.abspath("./data/interim/masked_images")
MASK_PATH = os.path.abspath("./resources/mask")
FIT_PATH = os.path.abspath("./data/interim/fit_images")

DETECTION_JSON = os.path.abspath("./resources/json/28570_images_0_003_threshold.json")

with open(DETECTION_JSON, "r") as f:
    result = json.load(f)

IMAGE_PATH = os.path.join(IMAGE_PATH, os.path.basename(DATASET_PATH))

MASK_PATH = os.path.join(MASK_PATH, os.path.basename(DATASET_PATH))

FIT_PATH = os.path.join(
    FIT_PATH,
    os.path.basename(DATASET_PATH),
)


print()
print(f"DATASET_PATH:   {DATASET_PATH}")
print(f"IMAGE_PATH:     {IMAGE_PATH}")
print(f"MASK_PATH:      {MASK_PATH}")
print(f"FIT_PATH:       {FIT_PATH}")
print(f"DETECTION_JSON: {DETECTION_JSON}")

In [ ]:
for image in tqdm.tqdm(result["images"]):
    image_file = os.path.join(IMAGE_PATH, image["file"])
    mask_file = os.path.join(MASK_PATH, image["file"])
    fit_file = os.path.join(FIT_PATH, image["file"])

    fi.fit_image(image_file, mask_file, fit_file)


In [ ]:
import os

from PIL import Image

# Ruta base
base_path = FIT_PATH

# Contadores para las imágenes que se pueden y no se pueden abrir
count_openable = 0
count_not_openable = 0

# Lista para almacenar las rutas de las imágenes que no se pueden abrir
not_openable_files = []

# Iterar sobre las rutas de las imágenes en el dataset
for image in result["images"]:
    file_path = image["file"]
    absolute_path = base_path + "/" + file_path
    try:
        # Intentar abrir la imagen
        img = Image.open(absolute_path)
        img.verify()  # Verificar que la imagen se puede abrir
        count_openable += 1
    except (IOError, SyntaxError) as e:
        # Si hay un error, incrementar el contador de imágenes no abiertas
        count_not_openable += 1
        not_openable_files.append(absolute_path)

# Imprimir los resultados
print(f"Number of openable images:     {count_openable}")
print(f"Number of not openable images: {count_not_openable}")
print(f"Not openable files:            {not_openable_files}")